In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"
import pandas as pd
import numpy as np

from quantfreedom.class_practice.enums import *
from quantfreedom.class_practice.base import backtest_df_only


np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

price_data = pd.read_hdf("../../tests/data/4hBTCETH.hd5")
%load_ext autoreload
%autoreload 2


In [64]:
# gonzalo : function definition
from plotly.subplots import make_subplots
from typing import List, Tuple
from scipy import stats
import numpy as np

def crossed_above_1d_nb(arr1, arr2, wait: int = 0):
    """Get the crossover of the first array going above the second array."""
    out = np.empty(arr1.shape, dtype=np.bool_)
    was_below = False
    crossed_ago = -1

    for i in range(arr1.shape[0]):
        if np.isnan(arr1[i]) or np.isnan(arr2[i]):
            crossed_ago = -1
            was_below = False
            out[i] = False
        elif arr1[i] > arr2[i]:
            if was_below:
                crossed_ago += 1
                out[i] = crossed_ago == wait
            else:
                out[i] = False
        elif arr1[i] == arr2[i]:
            crossed_ago = -1
            out[i] = False
        else:
            crossed_ago = -1
            was_below = True
            out[i] = False
    return out

def plot_candles_res_sup(data, rows, cols, support, resistance):
    fig = make_subplots(rows = rows, cols = cols, shared_xaxes=True, row_heights=[1000])
    fig.add_candlestick(
                    x=data.index.values,
                    open=data.Open.values,
                    high=data.High.values,
                    low=data.Low.values,
                    close=data.Close.values,
                    name='BTC',
                    row=1,
                    col=1)
    fig.update_layout(xaxis_rangeslider_visible=False)      # FIXME : only updates the rangeslider for the first row, to do the same with others, use4 xaxis2_rangeslider, xaxis3_rangeslider and so on...
    fig.add_scatter(x=data.index.values, y=support, mode='lines', line=dict(color='red'), name='Support', row=1, col=1)
    fig.add_scatter(x=data.index.values, y=resistance, mode='lines', line=dict(color='blue'), name='Resistance', row=1, col=1)

    return fig

def get_long_entries(close_values, support):
    in_range = np.where(np.isnan(support), 0, 1)
    return in_range & crossed_above_1d_nb(close_values, support)                 

def get_long_exit(close_values, resistance):
    in_range = np.where(np.isnan(resistance), 0, 1)
    return in_range & crossed_above_1d_nb(resistance, close_values)              

def strat_trade_within_range(fig, data, support, resistance):
    close = data.Close.values
    dates = data.index.values

    long_entries = get_long_entries(close, support)
    long_exit = get_long_exit(close, resistance)
    long_entries_display = np.where(long_entries, data.Close.values, np.nan)
    long_exit_display = np.where(long_exit, data.Close.values, np.nan)

    fig.add_scatter(x=dates, y=long_entries_display, mode='markers', marker=dict(size=20, symbol="arrow-up", color="Green"), name='long entry', row=1, col=1, secondary_y=False)
    fig.add_scatter(x=dates, y=long_exit_display, mode='markers', marker=dict(size=20, symbol="arrow-down", color="Red"), name='long exit', row=1, col=1, secondary_y=False)

def find_sweet_ranges(total_price_candles) -> List[Tuple[int,int,int,int]]:
    i = 1
    ci = 0.99
    min_chunk_size = 100
    max_sigma_pct_allowed = 0.10
    all_found_ranges : List[Tuple[int,int,int,int]] = []
    while i < len(total_price_candles):
        found_a_range = False

        current_mu = 0
        current_sigma = 0
        current_range_starting = i
        current_sweet_range = total_price_candles[-1*(i+min_chunk_size):-current_range_starting]
        res_mean, res_var, res_std = stats.bayes_mvs(current_sweet_range, alpha=ci)
        while i < len(total_price_candles) and res_var.statistic < (res_mean.statistic * max_sigma_pct_allowed):               # if N calculated have a variance lower than 'max_sigma_pct_allowed' from the mu calculated, then keep counting
            found_a_range = True
            current_mu = res_mean.statistic
            current_sigma = res_std.statistic
            i += 1

            # estimate parameters for the expanded range
            current_sweet_range = total_price_candles[-1*(i+min_chunk_size):-current_range_starting]
            res_mean, res_var, res_std = stats.bayes_mvs(current_sweet_range, alpha=ci)

        if found_a_range:
            all_found_ranges.append((current_range_starting, i+min_chunk_size-1, current_mu, current_sigma))

        i = i + 1       # FIXME : in the case the internal while gets to True, I might be advancing 'i' one extra time and therefore loosing one candle. 
        
    return all_found_ranges

def find_sourranding_bound(mu, sigma, confidence) -> Tuple[int,int]:
    return (stats.norm.ppf(confidence/100.0, mu, sigma), stats.norm.ppf(1-(confidence/100.0), mu, sigma))


def merge_ranges(price_candles : np.array, all_ranges : List[Tuple[int,int,int,int]]) -> Tuple[np.array, np.array]:
    lower_bound_values : np.array = np.array([np.nan] * len(price_candles))
    upper_bound_values : np.array = np.array([np.nan] * len(price_candles))
    for interval in all_ranges:
        bound_limits : Tuple[int,int] = find_sourranding_bound(interval[2], interval[3], 5)
        
        lower_bound_values[interval[0]:interval[1]] = bound_limits[0]
        upper_bound_values[interval[0]:interval[1]] = bound_limits[1]

    return (lower_bound_values[::-1], upper_bound_values[::-1])


In [60]:
import ccxt
exchange = ccxt.bitget(
    {
        'apiKey': 'bg_db47784d11a9a8e3d0e1ea2af8333d7f',
        'secret': '7b4b1e135abbb442210888603f520e607b2cb6476c4b9cfd466c61d735082b8d',
        'password': 'passphrasesnake1942'
    },
)
#exchange.set_sandbox_mode(True)
exchange.set_sandbox_mode(False)
#exchange.options['defaultType'] = 'swap'
exchange.load_markets()
symbol = "SBTC/SUSDT:SUSDT"
symbol = "BTC/USDT:USDT"
data_candles = exchange.fetch_ohlcv(symbol, limit=1000, params={'limit':'1000', 'after':'1692103157000', 'period':'5min'})

In [61]:
print(f'first:{data_candles[0][0]}, last:{data_candles[-1][0]}')

first:1694722620000, last:1694782500000


In [62]:
data = pd.DataFrame(data_candles, columns=['open_time', 'Open', 'High', 'Low', 'Close', 'Volume'])
data['open_time'] = data.apply(lambda row : pd.to_datetime(row['open_time'], unit='ms').strftime('%m/%d/%Y %H:%M:%S'), axis = 1)
data.set_index('open_time', inplace=True)
data.drop(columns=['Volume'], inplace=True)
data

,Open,High,Low,Close
open_time,,,,
09/14/2023 20:17:00,"26,590.10","26,595.40","26,588.80","26,589.00"
09/14/2023 20:18:00,"26,589.00","26,589.10","26,578.60","26,586.60"
09/14/2023 20:19:00,"26,586.60","26,611.60","26,586.60","26,603.50"
09/14/2023 20:20:00,"26,603.50","26,617.40","26,593.70","26,600.90"
09/14/2023 20:21:00,"26,600.90","26,600.90","26,589.50","26,599.30"
...,...,...,...,...
09/15/2023 12:51:00,"26,381.30","26,396.10","26,381.20","26,396.00"
09/15/2023 12:52:00,"26,396.00","26,396.00","26,385.50","26,385.50"
09/15/2023 12:53:00,"26,385.50","26,387.50","26,377.00","26,378.30"


In [65]:
ranges = find_sweet_ranges(data.Close.values)
data_in_order = data[::-1]
(support, resistance) = merge_ranges(data.Close.values, ranges)
fig = plot_candles_res_sup(data, 1, 1, support, resistance)
strat_trade_within_range(fig, data, support, resistance)
fig.show()

In [6]:
# gonzalo data lookup and signals definition
#data = vbt.YFData.download('BTC-USD', start='2023-08-29 UTC', end='2023-08-31 UTC', interval='5m')          # TODO : use ccxt to grab the data

ranges = find_sweet_ranges(data.Close.values)
(support, resistance) = merge_ranges(data.Close.values, ranges)

entries_array = get_long_entries(data.Close.values, support)
entries = pd.DataFrame(entries_array, index=data.index)

exit_signals = get_long_exit(data.Close.values, resistance)
exit_signals = pd.DataFrame(exit_signals, index=data.index)



In [24]:
exit_signals

,0
Timestamp,
1694746800000,0
1694747100000,0
1694747400000,0
1694747700000,0
1694748000000,0
...,...
1694775300000,0
1694775600000,0
1694775900000,0


In [8]:
# gonzalo
account_state = AccountState()
backtest_settings = BacktestSettings()
exchange_settings = ExchangeSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01]) / 100,
    sl_based_on_lookback=np.array([30]),
    risk_reward=np.array([1.0]),
    leverage_type=np.array([LeverageType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitType.Provided]),
    max_equity_risk_pct=np.array([3.0]) / 100,
    order_type=np.array([OrderType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.High]),
    sl_to_be_when_pct_from_candle_body=np.array([1, 2]) / 100,
    sl_to_be_zero_or_entry=np.array([SLToBeZeroOrEntryType.ZeroLoss]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.Close]),
    trail_sl_when_pct_from_candle_body=np.array([2.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([1.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
)

In [10]:
price_data = pd.read_hdf("../../tests/data/4hBTCETH.hd5")

In [22]:
backtest_df_only(
    account_state=account_state,
    order_settings_arrays=order_settings_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=exchange_settings,
    price_data=price_data,
    entries=entries,
    exit_signals=exit_signals,
)


Creating cartesian product ... after this the backtest will start, I promise :).



ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().